In [1]:
%load_ext autoreload
%autoreload 2
    
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # set your cuda device
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import torch
import tla
from tla import utils
from transformers import AutoModelForCausalLM, AutoTokenizer, LogitsProcessorList

device = 'cuda'

In [2]:
tokenizer = AutoTokenizer.from_pretrained("allenai/tulu-2-7b")
model = AutoModelForCausalLM.from_pretrained("allenai/tulu-2-7b").to(device)

/home/bazaluk/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/bazaluk/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
vocab_size = tokenizer.vocab_size

dfa = {
    "edges": [
    (0, 2, utils.populate_edge(['X', 'Y'], vocab_size, tokenizer)),
    (0, 1, utils.populate_edge(['V'], vocab_size, tokenizer)),
    (1, 2, utils.populate_edge(['1', '2', '3', '4', '5'], vocab_size, tokenizer)),
    (2, 3, utils.populate_edge(['->'], vocab_size, tokenizer)),
    (3, 4, utils.populate_edge(['V'], vocab_size, tokenizer)),
    (3, 5, utils.populate_edge(['X','Y'], vocab_size, tokenizer)),
    (4, 5, utils.populate_edge(['1', '2', '3', '4', '5'], vocab_size, tokenizer)),
    (5, 0, utils.populate_edge([','], vocab_size, tokenizer)),
    ],
    "initial_state": 0,
    "accept_states": set([5]),
    "current_state": 0,
}

In [3]:
vocab_size = tokenizer.vocab_size
import pandas as pd
data = pd.read_csv('/media/data/bazaluk/data_pairs_gen.csv')
data['pred_graphs'] = ''

prefix = data['prefix'].iloc[2000]
d_prompt = data['prompt'].iloc[2000]
#prefix='output a sequence of "A"s: '
#d_prompt='output one sequence with the letter "A". For example: "AAAA"'
suffix = '</s>'
soft_constraint = '' # use empty string for no soft constraint
prompt = f'<|user|>\n"{d_prompt}<|endoftext|>"{soft_constraint}:\n{prefix}\n<|assistant|>\n'


prefix_ids = tokenizer.encode(prefix)[1:]
suffix_ids = tokenizer.encode(suffix)[1:]
prompt_ids = tokenizer.encode(prompt)

In [11]:
#custom_logits_processor = utils.CustomLogitsProcessor(dfa)

min_new_tokens=1
max_new_tokens=40
eos_token_id = tokenizer.encode('<\s>')[1]
# set the hmm_batch_size & temperature
beam_size = 8 # sample 128 sequences
temperature = 0.7
prompt_size = len(prompt_ids)

custom_logits_processor = utils.ComplexLogitsProcessor(vocab_size, beam_size, prompt_size)

# generate with sampling, temperature=0.7
input_ids = torch.tensor([prompt_ids], device=device)
outputs = model.generate(
        input_ids=input_ids, do_sample=True,
        num_return_sequences=beam_size, 
        min_new_tokens=min_new_tokens, max_new_tokens=max_new_tokens,
        logits_processor=LogitsProcessorList([custom_logits_processor]),
        pad_token_id=eos_token_id,
        renormalize_logits=True,
    )

generated_ids = tla.extract_generated_ids(outputs.tolist(), prompt_ids, suffix_ids, eos_token_id)
print(tokenizer.decode(generated_ids[0]))
print('---')
print(tokenizer.decode(generated_ids[1]))
print('---')
print(tokenizer.decode(generated_ids[2]))
#print(tokenizer.decode(generated_ids[2]))

X -> V2 , V2 -> Y
---
X -> V1 , V1 -> Y , V2 -> V1 , V2 -> V2 , V1 -> V2 , Y -> Y
---
X -> V2


In [24]:
data['step1'].iloc[2000]

'V1->X,V1->Y,X->Y'

In [16]:
len(prompt_ids)

186

In [32]:
data['prompt'].iloc[1000]

'Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. Use V1 to represent gender. Use X to represent smoking. Use V3 to represent tar deposit. Use Y to represent lung cancer. The diagram should simply consist of edges denoted in "var1 -> var2" format, separated by commas.'

In [52]:
data = pd.read_csv('/media/data/bazaluk/data_pairs_gen.csv')
data['pred_graphs'] = ''

df=pd.DataFrame(columns=['prompt','prefix','step1','pred_graphs'])
df.to_csv('/media/data/bazaluk/token_look_ahead/data.csv')

for i in range(10):
    prefix = data['prefix'].iloc[i]
    d_prompt = data['prompt'].iloc[i]
    #prefix='output a sequence of "A"s: '
    #d_prompt='output one sequence with the letter "A". For example: "AAAA"'
    suffix = '</s>'
    soft_constraint = '' # use empty string for no soft constraint
    prompt = f'<|user|>\n"{d_prompt}<|endoftext|>"{soft_constraint}:\n{prefix}\n<|assistant|>\n'

    prefix_ids = tokenizer.encode(prefix)[1:]
    suffix_ids = tokenizer.encode(suffix)[1:]
    prompt_ids = tokenizer.encode(prompt)

    custom_logits_processor = utils.ComplexLogitsProcessor(vocab_size)

    min_new_tokens=1
    max_new_tokens=40
    eos_token_id = tokenizer.encode('<\s>')[1]
    # set the hmm_batch_size & temperature
    beam_size = 8 # sample 128 sequences
    temperature = 0.7

    # generate with sampling, temperature=0.7
    input_ids = torch.tensor([prompt_ids], device=device)
    outputs = model.generate(
        input_ids=input_ids, do_sample=True,
        num_return_sequences=beam_size, 
        min_new_tokens=min_new_tokens, max_new_tokens=max_new_tokens,
        logits_processor=LogitsProcessorList([custom_logits_processor]),
        pad_token_id=eos_token_id,
        renormalize_logits=True,
    )

    generated_ids = tla.extract_generated_ids(outputs.tolist(), prompt_ids, suffix_ids, eos_token_id)

    data.at[i,'pred_graphs'] = tokenizer.decode(generated_ids[0])
    data[['Unnamed: 0','prompt','prefix','step1','pred_graphs']][i:i+1].to_csv('/media/data/bazaluk/token_look_ahead/data.csv', mode='a', index=False, header=False)

In [53]:
pd.read_csv('/media/data/bazaluk/token_look_ahead/data.csv')

,Unnamed: 0,prompt,prefix,step1,pred_graphs
0,0,For patients who are young and pay a low hospi...,Identify the causal graph that depicts the re...,"V1->X,V1->Y,X->Y","V1 -> X ,"
1,1,The overall probability of citrus intake is 2%...,Identify the causal graph that depicts the re...,"X->V2,V2->Y",X -> V2
2,2,We know that liking spicy food and the chain-s...,Identify the causal graph that depicts the re...,"X->Y,V2->Y",V2 -> X
3,3,"For nonsmokers, the probability of freckles is...",Identify the causal graph that depicts the re...,"X->V2,X->Y,V2->Y",X -> V2
4,4,For individuals who are not male and applicant...,Identify the causal graph that depicts the re...,"X->V3,V2->V3,X->Y,V2->Y,V3->Y","V2 -> X1 ,"
5,5,"For farms with reduced crop yield per acre, th...",Identify the causal graph that depicts the re...,"V1->X,V2->X,V1->Y,X->Y",X1 -> V2
6,6,We know that CEO's decision to fire the employ...,Identify the causal graph that depicts the re...,"V1->V3,V1->X,X->Y,V3->Y",V1 -> V3
7,7,"For unvaccinated individuals, the probability ...",Identify the causal graph that depicts the re...,"X->V3,X->V2,V2->Y,V3->Y",V -> V3
8,8,We know that solar eclipse and heavy traffic c...,Identify the causal graph that depicts the re...,"X->Y,V2->Y",V2 -> Y
9,9,The overall probability of smoking is 81%. For...,Identify the causal graph that depicts the re...,"X->V2,V2->Y",X -> V1
